Przygotowanie danych 

In [1]:
import gzip
import numpy as np
import time

In [2]:
IMAGE_SIZE = 28

validation_image = gzip.open("C:\\Users\\rafal.000\\python\\allegro\\logistic_regression\\validate\\t10k-images-idx3-ubyte.gz", "r")
validation_labels = gzip.open("C:\\Users\\rafal.000\\python\\allegro\\logistic_regression\\validate\\t10k-labels-idx1-ubyte.gz", "r")

train_image = gzip.open("C:\\Users\\rafal.000\\python\\allegro\\logistic_regression\\train\\train-images-idx3-ubyte.gz", "r")
train_labels = gzip.open("C:\\Users\\rafal.000\\python\\allegro\\logistic_regression\\train\\train-labels-idx1-ubyte.gz", "r")

validation_image.read(16)
validation_labels.read(8)
train_image.read(16)
train_labels.read(8)

def is_prime(x):
    if x in [2,3,5,7]:
        return 1
    elif x in [4,6,8]:
        return 0 
    return -1

is_prime = np.vectorize(is_prime)

def generate_matrices(buffer, length):
    M = np.frombuffer(buffer, dtype=np.uint8)
    M = M.reshape(int(M.shape[0]/(length**2)), -1)

    return M

def generate_map(labels):
    map = np.argwhere(labels == -1)
    return map.T[0]


X = generate_matrices(train_image.read(), IMAGE_SIZE)
y = is_prime(generate_matrices(train_labels.read(), 1))
map = generate_map(y)

X = np.delete(X, map, 0)
y = np.delete(y, map, 0)



v_X = generate_matrices(validation_image.read(), IMAGE_SIZE)
v_y = is_prime(generate_matrices(validation_labels.read(), 1))
map = generate_map(v_y)

v_X = np.delete(v_X, map, 0)
v_y = np.delete(v_y, map, 0)

print(X.shape)
print(y.shape)



(41386, 784)
(41386, 1)


Model

In [54]:
theta = np.ndarray((IMAGE_SIZE **2, 1))
step_size = 0.8
epochs = 30
momentum = 0.9
reg = 0.00001

np.random.seed(1337)

def sigma(z):
    return 1 / (1 + np.exp(-z))

def cost(theta, X, y):
    cost = y*np.log(sigma(np.dot(X,theta)) + 1e-7)+(1-y)*np.log(1-sigma(np.dot(X,theta)) +1e-7)
    return cost.mean()
    
    
def fit(X, y):
    theta = np.zeros((IMAGE_SIZE **2, 1))
    V = np.zeros((IMAGE_SIZE **2, 1))

    start = time.time()
    iteration_start = start
    permutation = np.arange(X.shape[0])
    for epoch in range(1, epochs+1):
        np.random.shuffle(permutation)
        for i in permutation:
            gradient = (y[i] - sigma(np.matmul(theta.T, X[i])))*X[i].reshape(IMAGE_SIZE**2, 1) - 2*reg*theta
            V = momentum * V + (1-momentum) * gradient
            theta = theta + step_size*V
       
        print(f"{epoch}/{epochs} iteracja zakonczona po {time.time() - iteration_start}")
        iteration_start = time.time()
    print(f"trening zakonczony po {time.time() - start}")
    return theta

def predict(theta, X):
    return sigma(theta.T @ X)

def evaluate(theta, X, y):
    correct = 0.0
    total = 0.0 
    
    for i in range(X.shape[0]):
        if np.rint(predict(theta, X[i])) == y[i] : 
            correct += 1 
        total += 1 
    return correct/total


In [55]:
print(cost(theta, X, y))
theta = fit(X, y)
print(cost(theta, X, y))

-6.858739190342799
1/30 iteracja zakonczona po 1.4600822925567627
2/30 iteracja zakonczona po 1.5507605075836182
3/30 iteracja zakonczona po 1.4707624912261963
4/30 iteracja zakonczona po 1.6771645545959473
5/30 iteracja zakonczona po 1.4011597633361816
6/30 iteracja zakonczona po 1.576566219329834
7/30 iteracja zakonczona po 1.460557460784912
8/30 iteracja zakonczona po 1.5168743133544922
9/30 iteracja zakonczona po 1.4167296886444092
10/30 iteracja zakonczona po 1.4092230796813965
11/30 iteracja zakonczona po 1.5223913192749023
12/30 iteracja zakonczona po 1.4187994003295898
13/30 iteracja zakonczona po 1.5579898357391357
14/30 iteracja zakonczona po 1.398751974105835
15/30 iteracja zakonczona po 1.5687742233276367
16/30 iteracja zakonczona po 1.3984386920928955
17/30 iteracja zakonczona po 1.4078989028930664
18/30 iteracja zakonczona po 1.5678627490997314
19/30 iteracja zakonczona po 1.406402349472046
20/30 iteracja zakonczona po 1.5652697086334229
21/30 iteracja zakonczona po 1.409

In [56]:
print(evaluate(theta, X, y))
print(evaluate(theta, v_X, v_y))

0.8964384091238583
0.8945607911576497
